In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample-lodistic/Sample_Longlist_Data (2).xlsx


# THE TASK: ## Identification of promising e-commerce acquisition targets

### *Your task is to sanitize and analyze the data to profile the sellers present and develop selection criteria to identify the best or most promising sellers in this dataset, that the Acquisitions team at Razor should reach out to, and acquire* ##

# Import the dataset and take a peek

In [2]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 6.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
data = pd.read_excel('../input/sample-lodistic/Sample_Longlist_Data (2).xlsx')

data.head()

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL
0,2020-11-15,Garden,Seller 1,Seller 1-link,Seller 1-storefrontlink,"1-16 of over 100,000 results",88% positive in the last 12 months (118 ratings),Lohas Living Inc James Mazzello US 845 3RD Ave...,Business Name:Lohas Living Inc,411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-...,16,17,11,12,5873,4957,Visit the YAMAZAKI Store,https://www.amazon.de/-/en/stores/YAMAZAKI/pag...
1,2020-11-15,Garden,Seller 2,Seller 2-link,Seller 2-storefrontlink,NaN,90% positive in the last 12 months (566 ratings),Herzlich Willkommen im Amazon Shop von 1a-Hand...,Business Name:1a-Handelsagentur,1a HandelsagenturLindenallee 2MalchowMecklenbu...,0,2,2,9,0,0,Visit the tesa Store,https://www.amazon.de/-/en/stores/tesa/page/12...
2,2020-11-15,Garden,Seller 3,Seller 3-link,Seller 3-storefrontlink,"1-16 of over 2,000 results",85% positive in the last 12 months (75 ratings),67878u6i7 is committed to providing each custo...,Business Name:nantongaohuanfangzhipinyouxiango...,zhongnanshijicheng27zhuang1410haochongchuanqun...,15,0,25,12,164,162,TXYFYP,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...
3,2020-11-15,Garden,Seller 4,Seller 4-link,Seller 4-storefrontlink,1-16 of 123 results,NaN,7 days home life supermarket is committed to p...,Business Name:Zhengzhou Yaoqiong Trading Company,河南省中兴南路福禄街自贸试验区郑州片区（郑东）河南商会大厦1901室450000CN,1,0,0,0,0,0,SHIFENG,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...
4,2020-11-15,Garden,Seller 5,Seller 5-link,Seller 5-storefrontlink,"1-16 of over 1,000 results",81% positive in the last 12 months (52 ratings),78 68yui678 is committed to providing each cus...,Business Name:shenzhenfeichangshangmeimaoyiyou...,minzhidadaoshaxialaocun94dongA901longhuaxinqus...,16,0,31,15,224,113,YUOKI99,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...


In [4]:
data.shape

(1839, 18)

# 2. Filter out the sellers from China

In [5]:
business_loc = data['businessaddress']

In [6]:
business_loc.fillna('CN', inplace=True)

In [7]:
business_loc

0       411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-...
1       1a HandelsagenturLindenallee 2MalchowMecklenbu...
2       zhongnanshijicheng27zhuang1410haochongchuanqun...
3              河南省中兴南路福禄街自贸试验区郑州片区（郑东）河南商会大厦1901室450000CN
4       minzhidadaoshaxialaocun94dongA901longhuaxinqus...
                              ...                        
1834    Longhuaxinqu Longhuajiedao YousongDiergongyequ...
1835    hebeishengshijiazhuangshiwujixiangaotouxiangxi...
1836    lankaoxianzhangjunmuzhenzhengzhuangcunerzukaif...
1837    bei san huan da ming gong jian cai shi changba...
1838    honggutanxinquhuanandongerlu99haolongxiangyuan...
Name: businessaddress, Length: 1839, dtype: object

In [8]:
chinese_sellers = business_loc.apply(lambda x: 'CN' not in x)
chinese_sellers

0        True
1        True
2       False
3       False
4       False
        ...  
1834    False
1835    False
1836    False
1837    False
1838    False
Name: businessaddress, Length: 1839, dtype: bool

In [9]:
data = data[chinese_sellers]

In [10]:
data.head()

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL
0,2020-11-15,Garden,Seller 1,Seller 1-link,Seller 1-storefrontlink,"1-16 of over 100,000 results",88% positive in the last 12 months (118 ratings),Lohas Living Inc James Mazzello US 845 3RD Ave...,Business Name:Lohas Living Inc,411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-...,16,17,11,12,5873,4957,Visit the YAMAZAKI Store,https://www.amazon.de/-/en/stores/YAMAZAKI/pag...
1,2020-11-15,Garden,Seller 2,Seller 2-link,Seller 2-storefrontlink,NaN,90% positive in the last 12 months (566 ratings),Herzlich Willkommen im Amazon Shop von 1a-Hand...,Business Name:1a-Handelsagentur,1a HandelsagenturLindenallee 2MalchowMecklenbu...,0,2,2,9,0,0,Visit the tesa Store,https://www.amazon.de/-/en/stores/tesa/page/12...
14,2020-11-15,Garden,Seller 15,Seller 15-link,Seller 15-storefrontlink,1-16 of 919 results,97% positive in the last 12 months (116 ratings),Impressum\n \n Gesetzliche Anbieterkennung:\n ...,Business Name:Oliver Mills,Kaiserstr.12Borgholzhausen33829DE,15,0,0,3,1446,921,Unbekannt,https://www.amazon.de/-/en/Unbekannt/b/ref=bl_...
19,2020-11-15,Garden,Seller 20,Seller 20-link,Seller 20-storefrontlink,NaN,76% positive in the last 12 months (3721 ratings),Impressum: ABC-Schnäppchenmarkt GmbH \n Jösser...,Business Name:ABC-Schnäppchenmarkt GmbH,Jösser Weg10Petershagen32469DE,0,18,20,20,0,0,Visit the TFA Dostmann Store,https://www.amazon.de/-/en/stores/TFA+Dostmann...
20,2020-11-15,Garden,Seller 21,Seller 21-link,Seller 21-storefrontlink,1-16 of 123 results,NaN,AllSparesEire is committed to providing each c...,Business Name:All Terrain Ireland,WEXFORDGOREY46 MAIN STY25T2N4IE,14,0,0,0,837,787,Mountfield Outdoor Spares,https://www.amazon.de/-/en/Mountfield-Outdoor-...


In [11]:
data.shape

(525, 18)

# 3. Make a dataframe with relevant columns for seller selection exercise

In [12]:
data_analysis = data[['sellerproductcount', 'sellerratings', 'Count of seller brands', 'Max % of negative seller ratings - last 12 months', 'Hero Product 1 #ratings', 'Hero Product 2 #ratings', 'sellerdetails']]

In [13]:
data_analysis.head()

,sellerproductcount,sellerratings,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails
0,"1-16 of over 100,000 results",88% positive in the last 12 months (118 ratings),16,12,5873,4957,Lohas Living Inc James Mazzello US 845 3RD Ave...
1,NaN,90% positive in the last 12 months (566 ratings),0,9,0,0,Herzlich Willkommen im Amazon Shop von 1a-Hand...
14,1-16 of 919 results,97% positive in the last 12 months (116 ratings),15,3,1446,921,Impressum\n \n Gesetzliche Anbieterkennung:\n ...
19,NaN,76% positive in the last 12 months (3721 ratings),0,20,0,0,Impressum: ABC-Schnäppchenmarkt GmbH \n Jösser...
20,1-16 of 123 results,NaN,14,0,837,787,AllSparesEire is committed to providing each c...


# 4. Check for NaN values and drop those rows

In [14]:
data_analysis.isna().sum()

sellerproductcount                                   89
sellerratings                                         6
Count of seller brands                                0
Max % of negative seller ratings - last 12 months     0
Hero Product 1 #ratings                               0
Hero Product 2 #ratings                               0
sellerdetails                                         2
dtype: int64

In [15]:
data_analysis.dropna(inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [16]:
data_analysis.head()

,sellerproductcount,sellerratings,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails
0,"1-16 of over 100,000 results",88% positive in the last 12 months (118 ratings),16,12,5873,4957,Lohas Living Inc James Mazzello US 845 3RD Ave...
14,1-16 of 919 results,97% positive in the last 12 months (116 ratings),15,3,1446,921,Impressum\n \n Gesetzliche Anbieterkennung:\n ...
22,"1-16 of over 40,000 results",100% positive in the last 12 months (10 ratings),16,0,10538,8293,Bestof Floral is committed to providing each c...
23,"1-16 of over 100,000 results",88% positive in the last 12 months (3134 ratings),15,10,30139,11182,Blumenbecker Industriebedarf GmbH Sudhoferweg ...
24,"1-16 of over 3,000 results",97% positive in the last 12 months (610 ratings),15,2,3286,3079,Herzlich willkommen auf unserer Internetpräsen...


In [17]:
data_analysis.shape

(431, 7)

# 5. Use Regular Expressions to parse out useful numeric values from the sellerproductcount and sellerratings columns

In [18]:
import regex as re

In [19]:
string1 =  '1-16 of over 100,000 results'
string1

'1-16 of over 100,000 results'

#### Here we will extract the total product count

In [20]:
match = re.findall(r' (\d*.\d+) results', string1)
match

['100,000']

In [21]:
data_analysis['sellerproductcount'] = data_analysis['sellerproductcount'].apply(lambda x: ''.join(re.findall(r' (\d*.\d+) results', x)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Check the result

In [22]:
data_analysis['sellerproductcount']

0       100,000
14          919
22       40,000
23      100,000
24        3,000
         ...   
1784    100,000
1785        285
1808         52
1814      1,000
1815      2,000
Name: sellerproductcount, Length: 431, dtype: object

In [23]:
data_analysis['sellerproductcount'] = data_analysis['sellerproductcount'].apply(lambda x: x.replace(',',''))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


##### The below function will help make the strings into numeric values

In [24]:
def num(x):
    try:
        return int(x)
    except:
        return 0

In [25]:
data_analysis['sellerproductcount'] = data_analysis['sellerproductcount'].apply(num)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Check the result

In [26]:
data_analysis['sellerproductcount'].tail(20)

1730     40000
1761      2000
1762       284
1765       184
1766      2000
1767       643
1770       347
1771      1000
1773      4000
1775       235
1776       442
1778        33
1779       211
1780      1000
1781       400
1784    100000
1785       285
1808        52
1814      1000
1815      2000
Name: sellerproductcount, dtype: int64

#### Peek the dataframe

In [27]:
data_analysis.head()

,sellerproductcount,sellerratings,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails
0,100000,88% positive in the last 12 months (118 ratings),16,12,5873,4957,Lohas Living Inc James Mazzello US 845 3RD Ave...
14,919,97% positive in the last 12 months (116 ratings),15,3,1446,921,Impressum\n \n Gesetzliche Anbieterkennung:\n ...
22,40000,100% positive in the last 12 months (10 ratings),16,0,10538,8293,Bestof Floral is committed to providing each c...
23,100000,88% positive in the last 12 months (3134 ratings),15,10,30139,11182,Blumenbecker Industriebedarf GmbH Sudhoferweg ...
24,3000,97% positive in the last 12 months (610 ratings),15,2,3286,3079,Herzlich willkommen auf unserer Internetpräsen...


In [28]:
string2 = '88% positive in the last 12 months (31374 ratings)'
string2

'88% positive in the last 12 months (31374 ratings)'

#### Here we will extract the percent positive rating

In [29]:
re.findall(r'(\d+)%', string2)

['88']

In [30]:
re.findall(r'(\d+) ratings', string2)

['31374']

In [31]:
data_analysis['sellerratings%positive'] = data_analysis['sellerratings'].apply(lambda x: int(re.findall(r'(\d+)%', x)[0]))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
data_analysis['sellerratingstotalcount'] = data_analysis['sellerratings'].apply(lambda x: int(re.findall(r'\((\d+)', x)[0]))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Peek the data

In [33]:
data_analysis.head()

,sellerproductcount,sellerratings,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails,sellerratings%positive,sellerratingstotalcount
0,100000,88% positive in the last 12 months (118 ratings),16,12,5873,4957,Lohas Living Inc James Mazzello US 845 3RD Ave...,88,118
14,919,97% positive in the last 12 months (116 ratings),15,3,1446,921,Impressum\n \n Gesetzliche Anbieterkennung:\n ...,97,116
22,40000,100% positive in the last 12 months (10 ratings),16,0,10538,8293,Bestof Floral is committed to providing each c...,100,10
23,100000,88% positive in the last 12 months (3134 ratings),15,10,30139,11182,Blumenbecker Industriebedarf GmbH Sudhoferweg ...,88,3134
24,3000,97% positive in the last 12 months (610 ratings),15,2,3286,3079,Herzlich willkommen auf unserer Internetpräsen...,97,610


#### Now drop the sellerratings column as we have extracted useful information into new columns

In [34]:
data_analysis.drop('sellerratings', axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### Take a peek again

In [35]:
data_analysis.head()

,sellerproductcount,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails,sellerratings%positive,sellerratingstotalcount
0,100000,16,12,5873,4957,Lohas Living Inc James Mazzello US 845 3RD Ave...,88,118
14,919,15,3,1446,921,Impressum\n \n Gesetzliche Anbieterkennung:\n ...,97,116
22,40000,16,0,10538,8293,Bestof Floral is committed to providing each c...,100,10
23,100000,15,10,30139,11182,Blumenbecker Industriebedarf GmbH Sudhoferweg ...,88,3134
24,3000,15,2,3286,3079,Herzlich willkommen auf unserer Internetpräsen...,97,610


In [36]:
data_analysis.dtypes

sellerproductcount                                    int64
Count of seller brands                                int64
Max % of negative seller ratings - last 12 months     int64
Hero Product 1 #ratings                               int64
Hero Product 2 #ratings                               int64
sellerdetails                                        object
sellerratings%positive                                int64
sellerratingstotalcount                               int64
dtype: object

# 6. Data Manipulation to calculate seller total score with which we will rank them

#### The Max % Negative ratings can be converted to a positive attribute using 100 - % value

In [37]:
data_analysis['Max % of negative seller ratings - last 12 months'] = 100 - data_analysis['Max % of negative seller ratings - last 12 months']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Take a peek again

In [38]:
data_analysis.head()

,sellerproductcount,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails,sellerratings%positive,sellerratingstotalcount
0,100000,16,88,5873,4957,Lohas Living Inc James Mazzello US 845 3RD Ave...,88,118
14,919,15,97,1446,921,Impressum\n \n Gesetzliche Anbieterkennung:\n ...,97,116
22,40000,16,100,10538,8293,Bestof Floral is committed to providing each c...,100,10
23,100000,15,90,30139,11182,Blumenbecker Industriebedarf GmbH Sudhoferweg ...,88,3134
24,3000,15,98,3286,3079,Herzlich willkommen auf unserer Internetpräsen...,97,610


In [39]:
data_analysis.columns

Index(['sellerproductcount', 'Count of seller brands',
       'Max % of negative seller ratings - last 12 months',
       'Hero Product 1 #ratings', 'Hero Product 2 #ratings', 'sellerdetails',
       'sellerratings%positive', 'sellerratingstotalcount'],
      dtype='object')

#### We will scale the columns so that the score there values are between 0 and 1 and all columns are given equal importance in seller selection

In [40]:
from sklearn.preprocessing import MinMaxScaler

In [41]:
scalar = MinMaxScaler()

In [42]:
data_analysis[['sellerproductcount', 'Count of seller brands','Max % of negative seller ratings - last 12 months','Hero Product 1 #ratings', 'Hero Product 2 #ratings', 'sellerratings%positive', 'sellerratingstotalcount']] =  scalar.fit_transform(data_analysis[['sellerproductcount', 'Count of seller brands', 'Max % of negative seller ratings - last 12 months', 'Hero Product 1 #ratings', 'Hero Product 2 #ratings', 'sellerratings%positive', 'sellerratingstotalcount']])

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


#### Take a peek at the scaled values now. All values should range between 0 and 1

In [43]:
data_analysis.head()

,sellerproductcount,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails,sellerratings%positive,sellerratingstotalcount
0,0.500000,1.0000,0.88,0.067618,0.070081,Lohas Living Inc James Mazzello US 845 3RD Ave...,0.868132,0.015675
14,0.004595,0.9375,0.97,0.016648,0.013021,Impressum\n \n Gesetzliche Anbieterkennung:\n ...,0.967033,0.015407
22,0.200000,1.0000,1.00,0.121327,0.117245,Bestof Floral is committed to providing each c...,1.000000,0.001206
23,0.500000,0.9375,0.90,0.347000,0.158090,Blumenbecker Industriebedarf GmbH Sudhoferweg ...,0.868132,0.419748
24,0.015000,0.9375,0.98,0.037833,0.043531,Herzlich willkommen auf unserer Internetpräsen...,0.967033,0.081592


#### Create a new column which provides total score by mutiplying all the values

In [44]:
data_analysis['total_score'] = data_analysis['sellerproductcount'] * data_analysis['Count of seller brands'] * data_analysis['Hero Product 1 #ratings'] * data_analysis['Hero Product 2 #ratings']  * data_analysis['sellerratings%positive'] * data_analysis['sellerratingstotalcount']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Verify the total score column

In [45]:
data_analysis.head()

,sellerproductcount,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails,sellerratings%positive,sellerratingstotalcount,total_score
0,0.500000,1.0000,0.88,0.067618,0.070081,Lohas Living Inc James Mazzello US 845 3RD Ave...,0.868132,0.015675,3.224283e-05
14,0.004595,0.9375,0.97,0.016648,0.013021,Impressum\n \n Gesetzliche Anbieterkennung:\n ...,0.967033,0.015407,1.391350e-08
22,0.200000,1.0000,1.00,0.121327,0.117245,Bestof Floral is committed to providing each c...,1.000000,0.001206,3.430481e-06
23,0.500000,0.9375,0.90,0.347000,0.158090,Blumenbecker Industriebedarf GmbH Sudhoferweg ...,0.868132,0.419748,9.370188e-03
24,0.015000,0.9375,0.98,0.037833,0.043531,Herzlich willkommen auf unserer Internetpräsen...,0.967033,0.081592,1.827305e-06


#### Scale the total score column so that the values range between 0 for lowest score and 1 for best score

In [46]:
data_analysis['total_score'] = scalar.fit_transform(data_analysis[['total_score']])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Verify now

In [47]:
data_analysis.head()

,sellerproductcount,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails,sellerratings%positive,sellerratingstotalcount,total_score
0,0.500000,1.0000,0.88,0.067618,0.070081,Lohas Living Inc James Mazzello US 845 3RD Ave...,0.868132,0.015675,1.767296e-03
14,0.004595,0.9375,0.97,0.016648,0.013021,Impressum\n \n Gesetzliche Anbieterkennung:\n ...,0.967033,0.015407,7.626279e-07
22,0.200000,1.0000,1.00,0.121327,0.117245,Bestof Floral is committed to providing each c...,1.000000,0.001206,1.880318e-04
23,0.500000,0.9375,0.90,0.347000,0.158090,Blumenbecker Industriebedarf GmbH Sudhoferweg ...,0.868132,0.419748,5.135995e-01
24,0.015000,0.9375,0.98,0.037833,0.043531,Herzlich willkommen auf unserer Internetpräsen...,0.967033,0.081592,1.001584e-04


# 6. Make a new dataframe with Top 20 Best Seller with respectto their Total Score

In [48]:
top20_sellers = data_analysis.sort_values('total_score', ascending=False).head(20)

#### Look at the DataFrame

In [49]:
top20_sellers

,sellerproductcount,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails,sellerratings%positive,sellerratingstotalcount,total_score
36,0.250,0.9375,0.92,0.468396,0.267347,IMPRESSUM \n itenga GmbH\n Am Umspannwerk 2\n ...,0.901099,0.689845,1.000000
726,0.200,1.0000,0.91,0.468373,0.188189,trendmile GmbH\n\nAmtsgericht Darmstadt HR B 8...,0.879121,0.970123,0.824078
495,0.500,0.8750,0.93,0.894745,1.000000,SELECT ELITE is committed to providing each cu...,0.901099,0.038451,0.743424
23,0.500,0.9375,0.90,0.347000,0.158090,Blumenbecker Industriebedarf GmbH Sudhoferweg ...,0.868132,0.419748,0.513599
322,0.150,1.0000,0.94,0.825193,0.824690,Milovan Stojkovic\n Mühlenstr. 16\n 48431 Rhei...,0.912088,0.093516,0.477238
712,1.000,0.9375,0.78,0.155315,0.159532,TradeINN is committed to providing each custom...,0.736264,0.301447,0.282586
882,0.020,0.9375,0.96,0.768813,0.458350,Impressum\n \n Gesetzliche Anbieterkennung:\...,0.934066,0.815247,0.275779
100,0.035,1.0000,0.98,0.545006,0.458350,Kessler electronic GmbH\n Dieselstrasse 4\n 50...,0.967033,0.457529,0.212033
1784,0.500,1.0000,0.98,0.297066,0.070392,Impressum nach TMG:\n\nZoreno Versandhandel Gm...,0.978022,0.352224,0.197421
985,0.500,1.0000,0.96,0.468407,0.188189,"Schreibwaren Wegmann, Janos Metz e.K. Inhaber:...",0.934066,0.048499,0.109441


# 7. Extract the phone numbers and email addresses for the top 20 sellers

 #### Extract the email data

In [50]:
re.findall(r'\S+@\S+', top20_sellers['sellerdetails'][36])

['info@itenga.de']

In [51]:
top20_sellers['email'] = top20_sellers['sellerdetails'].apply(lambda x: ''.join(re.findall(r'\S+@\S+', x)))

In [52]:
top20_sellers

,sellerproductcount,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails,sellerratings%positive,sellerratingstotalcount,total_score,email
36,0.250,0.9375,0.92,0.468396,0.267347,IMPRESSUM \n itenga GmbH\n Am Umspannwerk 2\n ...,0.901099,0.689845,1.000000,info@itenga.de
726,0.200,1.0000,0.91,0.468373,0.188189,trendmile GmbH\n\nAmtsgericht Darmstadt HR B 8...,0.879121,0.970123,0.824078,support@trendmile.com
495,0.500,0.8750,0.93,0.894745,1.000000,SELECT ELITE is committed to providing each cu...,0.901099,0.038451,0.743424,
23,0.500,0.9375,0.90,0.347000,0.158090,Blumenbecker Industriebedarf GmbH Sudhoferweg ...,0.868132,0.419748,0.513599,bib.beckum@blumenbecker.com
322,0.150,1.0000,0.94,0.825193,0.824690,Milovan Stojkovic\n Mühlenstr. 16\n 48431 Rhei...,0.912088,0.093516,0.477238,info@limuno.com
712,1.000,0.9375,0.78,0.155315,0.159532,TradeINN is committed to providing each custom...,0.736264,0.301447,0.282586,
882,0.020,0.9375,0.96,0.768813,0.458350,Impressum\n \n Gesetzliche Anbieterkennung:\...,0.934066,0.815247,0.275779,service@vitamed-quinger.comDatenschutz@vitamed...
100,0.035,1.0000,0.98,0.545006,0.458350,Kessler electronic GmbH\n Dieselstrasse 4\n 50...,0.967033,0.457529,0.212033,info@kessler-electronic.de
1784,0.500,1.0000,0.98,0.297066,0.070392,Impressum nach TMG:\n\nZoreno Versandhandel Gm...,0.978022,0.352224,0.197421,ade.support@zoreno.com
985,0.500,1.0000,0.96,0.468407,0.188189,"Schreibwaren Wegmann, Janos Metz e.K. Inhaber:...",0.934066,0.048499,0.109441,info@schreibwaren-wegmann.de


#### Extract the telephone data

In [53]:
top20_sellers['sellerdetails'][36]

'IMPRESSUM \n itenga GmbH\n Am Umspannwerk 2\n 66557 Illingen\n Deutschland Tel: 06825 / 89697-0 Fax: 06825 / 89697-20 E-Mail: info@itenga.de Geschäftsführer: Thomas Meiser Registergericht: Amtsgericht Saarbrücken\n Registernummer: HRB 17443 Umsatzsteuer-Identifikationsnummer gemäß § 27 a Umsatzsteuergesetz: DE815026868 WEEE-Registrationsnummer: DE 56344043 \n Plattform der EU-Kommission zur Online-Streitbeilegung: ec.europa.eu/consumers/odr Wir sind zur Teilnahme an einem Streitbeilegungsverfahren vor einer...'

In [54]:
re.findall(r'Tel.+', top20_sellers['sellerdetails'][36])

['Tel: 06825 / 89697-0 Fax: 06825 / 89697-20 E-Mail: info@itenga.de Geschäftsführer: Thomas Meiser Registergericht: Amtsgericht Saarbrücken']

In [55]:
top20_sellers['telephone'] = top20_sellers['sellerdetails'].apply(lambda x: ' '.join(re.findall(r'Tel.+', x)))

In [56]:
top20_sellers

,sellerproductcount,Count of seller brands,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerdetails,sellerratings%positive,sellerratingstotalcount,total_score,email,telephone
36,0.250,0.9375,0.92,0.468396,0.267347,IMPRESSUM \n itenga GmbH\n Am Umspannwerk 2\n ...,0.901099,0.689845,1.000000,info@itenga.de,Tel: 06825 / 89697-0 Fax: 06825 / 89697-20 E-M...
726,0.200,1.0000,0.91,0.468373,0.188189,trendmile GmbH\n\nAmtsgericht Darmstadt HR B 8...,0.879121,0.970123,0.824078,support@trendmile.com,Telefon: 06206 – 951326 - 0 Telefax: 06206 – 9...
495,0.500,0.8750,0.93,0.894745,1.000000,SELECT ELITE is committed to providing each cu...,0.901099,0.038451,0.743424,,
23,0.500,0.9375,0.90,0.347000,0.158090,Blumenbecker Industriebedarf GmbH Sudhoferweg ...,0.868132,0.419748,0.513599,bib.beckum@blumenbecker.com,Telefon: 02521/8406-0 E-Mail: bib.beckum@blume...
322,0.150,1.0000,0.94,0.825193,0.824690,Milovan Stojkovic\n Mühlenstr. 16\n 48431 Rhei...,0.912088,0.093516,0.477238,info@limuno.com,Tel.: +49 59 71/ 8 00 38 03
712,1.000,0.9375,0.78,0.155315,0.159532,TradeINN is committed to providing each custom...,0.736264,0.301447,0.282586,,
882,0.020,0.9375,0.96,0.768813,0.458350,Impressum\n \n Gesetzliche Anbieterkennung:\...,0.934066,0.815247,0.275779,service@vitamed-quinger.comDatenschutz@vitamed...,Telefon: +4962369329325
100,0.035,1.0000,0.98,0.545006,0.458350,Kessler electronic GmbH\n Dieselstrasse 4\n 50...,0.967033,0.457529,0.212033,info@kessler-electronic.de,Telefon: + 49 (0)2273 / 991-9325 (werktags 9:0...
1784,0.500,1.0000,0.98,0.297066,0.070392,Impressum nach TMG:\n\nZoreno Versandhandel Gm...,0.978022,0.352224,0.197421,ade.support@zoreno.com,
985,0.500,1.0000,0.96,0.468407,0.188189,"Schreibwaren Wegmann, Janos Metz e.K. Inhaber:...",0.934066,0.048499,0.109441,info@schreibwaren-wegmann.de,Tel.: 09922609490 Fax: 09922609491 E-Mail: inf...


# 8. Final Table with Top 20 Sellers

In [57]:
top20_sellers.index

Int64Index([  36,  726,  495,   23,  322,  712,  882,  100, 1784,  985,   53,
            1495,  947,  321, 1058, 1045,   26,  386,  107,  656],
           dtype='int64')

In [58]:
pd.DataFrame(data['seller business name'][top20_sellers.index].values, index=range(1,21), columns=['Top 20 Seller Names'])

,Top 20 Seller Names
1,Business Name:itenga GmbH
2,Business Name:trendmile GmbH
3,Business Name:C. Y. MARKETING INC.
4,Business Name:Blumenbecker Industriebedarf GmbH
5,Business Name:Milovan Stojkovic
6,Business Name:TRADEINN RETAIL SERVICES SL
7,Business Name:Vitamed Matthias Quinger e.K.
8,Business Name:Kessler electronic GmbH
9,Business Name:Zoreno Versandhandel GmbH
10,"Business Name:Schreibwaren Wegmann, Inh. Janos..."


# The End